In [96]:
import numpy as np
from qiskit import *
from math import sqrt, pi
#preparing the state coeffs
a = np.random.rand(1) + 1j * np.random.rand(1)
b = np.random.rand(1) + 1j * np.random.rand(1)
tnorm = sqrt(np.abs(a)**2 + np.abs(b)**2)
a = a / tnorm
b = b / tnorm

psi = np.angle(a)
a = a * np.exp(-1j * psi)
b = b * np.exp(-1j * psi)
#print(a,b)

phi = np.angle(b)
theta  = 2 * np.arctan2(np.abs(b),np.abs(a))
print('theta = ',theta,', phi = ', phi)

theta =  [1.81343973] , phi =  [1.06014521]


In [99]:
with open('Qconfig_IBMQ_experience.py', 'r') as token_file:
    token = token_file.read()
qk.IBMQ.save_account(token, overwrite = True)
qk.IBMQ.load_account()
my_provider = IBMQ.get_provider()

qr = QuantumRegister(1)
cr = ClassicalRegister(1)
circuit = QuantumCircuit(qr, cr)

state0[0] = complex(a)
state0[1] = complex(b)
print(state0)
circuit.initialize(state0, qr)
circuit.h(0)
circuit.measure(qr,cr)

backend=Aer.get_backend('qasm_simulator')
jobIBM=execute(circuit,backend=backend,shots=1024)
resultSim=jobIBM.result()

backend  = my_provider.get_backend('ibmq_ourense')
jobIBM=execute(circuit,backend=backend,shots=1024)
resultExp=jobIBM.result()

from qiskit.tools.visualization import plot_histogram
#plot_histogram([resultSim.get_counts(circuit)])
plot_histogram([resultExp.get_counts(circuit),resultSim.get_counts(circuit)])

rsim = resultSim.get_counts(circuit)
a2sim = rsim['0'] / (rsim['0'] + rsim['1'])
cos_phi1 = (2*a2sim-1)/(np.sin(theta))
newphi1 = np.arccos(cos_phi1)

rexp = resultExp.get_counts(circuit)
a2exp = rexp['0'] / (rexp['0'] + rexp['1'])
cos_phi2 = (2*a2exp-1)/(np.sin(theta)) #the explanation is on google disk link
newphi2 = np.arccos(cos_phi2)
#due to cos being even, we can't get the sign of phi through a single circuit
print('real phi = ',phi, ', obtained phi = ',newphi1, newphi2)


C:\ProgramData\Anaconda3\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '
ibmqfactory.load_account:WARNING:2020-09-25 03:38:08,479: Credentials are already in use. The existing account in the session will be replaced.


[(0.6163321200365067+0j), (0.3848803191136271+0.687023913535983j)]
real phi =  [1.06014521] , obtained phi =  [1.09420723] [1.01790588]


In [65]:
circuit.draw()

┌──────────────────────────────────────┐┌───┐┌─┐
q101_0: ┤ initialize(0.78041,0.50259+0.37198j) ├┤ H ├┤M├
        └──────────────────────────────────────┘└───┘└╥┘
 c33: 1/══════════════════════════════════════════════╩═
                                                      0